## Basic classifier that uses Bidirectional Gated Recurrent Unit (bigru) to predict if the payload is an SQL injection

Let's import our stuff here


In [1]:
!pip3 install scikit-learn>=1.0.0
!pip3 install ktrain matplotlib tensorflow numpy
import matplotlib
import os
import numpy as np
%reload_ext autoreload
%autoreload 2
%matplotlib inline
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

Some more imports...We are using ktrain wrapper to simplify model operations and take advantage of some cool stuff like autofit

In [2]:
import ktrain
from ktrain import text

Here, we will classify payload to "malicious sqli" and "benign" . Data set is presented as a CSV file (i.e., download the file ```SQLiV3a_.csv```).  We will load the data using the ```texts_from_csv``` method, which assumes the label_columns are already one-hot-encoded in the spreadsheet. Since *val_filepath* is None, 10% of the data will automatically be used as a validation set.


In [3]:
DATA_PATH = 'trainlist_22k.csv'
NUM_WORDS = 50000
MAXLEN = 100
trn, val, preproc = text.texts_from_csv(DATA_PATH,
                      'payload',
                      label_columns = ["type"],
                      val_filepath=None, # if None, 10% of data will be used for validation
                      max_features=NUM_WORDS, maxlen=MAXLEN,
                      ngram_range=1,
                      preprocess_mode='distilbert')

detected encoding: UTF-8-SIG (if wrong, set manually)
['benign', 'sqli', 'xss']
       benign  sqli  xss
15374     1.0   0.0  0.0
10885     0.0   1.0  0.0
12800     1.0   0.0  0.0
18741     1.0   0.0  0.0
21681     0.0   0.0  1.0
['benign', 'sqli', 'xss']
       benign  sqli  xss
17764     0.0   0.0  1.0
3366      0.0   1.0  0.0
20793     0.0   0.0  1.0
8782      0.0   1.0  0.0
13607     1.0   0.0  0.0


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 9
	95percentile : 40
	99percentile : 79


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 43
	99percentile : 80


Load the learner instance that uses ```bigru``` model 

In [13]:
from tensorflow.keras.layers import Dense, GaussianDropout

def get_model():
    model = text.text_classifier('distilbert', (trn), 
                             preproc=preproc)
    #model.add(Dense(3, activation='sigmoid'))
    #model.add(GaussianDropout1D(0.2))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = get_model()
learner = ktrain.get_learner(model, train_data=(trn), val_data=(val))

Is Multi-Label? False
maxlen is 100
done.


Here is what our model looks like. It has a number of layers that comprise this ```Distilbert```

In [14]:
learner.print_layers()


0 (trainable=True) : <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertMainLayer object at 0x7f18fbd39f70>
1 (trainable=True) : <keras.layers.core.dense.Dense object at 0x7f18677dc220>
2 (trainable=True) : <keras.layers.core.dense.Dense object at 0x7f18677dc520>
3 (trainable=True) : <keras.layers.core.dropout.Dropout object at 0x7f18677dc7f0>


In [15]:
learner.freeze(1)

In [ ]:
text.print_text_classifiers()

Now let's find the optimal learning rate for this model

In [11]:
learner.lr_find(show_plot=True, max_epochs=2)

simulating training for different learning rates... this may take a few moments...
Epoch 1/2
 14/631 [..............................] - ETA: 9:08 - loss: 2.9296 - accuracy: 0.3705

Train the model using the optimal learning rate ( adjust argument as needed accorfing to the graph)

In [16]:
learner.fit_onecycle(3e-5, 4)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
632/632 [==============================] - 570s 892ms/step - loss: 0.8786 - accuracy: 0.5829 - val_loss: 0.3634 - val_accuracy: 0.9408
Epoch 2/4
632/632 [==============================] - 562s 887ms/step - loss: 0.1859 - accuracy: 0.9553 - val_loss: 0.0732 - val_accuracy: 0.9746
Epoch 3/4
632/632 [==============================] - 560s 883ms/step - loss: 0.0622 - accuracy: 0.9772 - val_loss: 0.0489 - val_accuracy: 0.9835
Epoch 4/4
632/632 [==============================] - 555s 875ms/step - loss: 0.0493 - accuracy: 0.9829 - val_loss: 0.0444 - val_accuracy: 0.9853


In [ ]:
#learner.autofit(0.001)

It's time for some fun! First, get a predictor instance that uses our pre-trained model

In [17]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('detector_model_distilbert')
print('MODEL SAVED')

MODEL SAVED


Let's see if it can catch an SQLi or XSS payload

In [22]:
predictor = ktrain.load_predictor('/home/jupyter/Jupyter/notebook/ml_sqli_detector/predictor_distilbert')
new_model = ktrain.get_predictor(predictor.model, predictor.preproc)
#new_model = load_model('detector_model_bert/tf_model.h5')
labels = ['benign', 'sqli', 'xss']

In [26]:
text = '<applet onkeydown="alert(1)" contenteditable>test</applet>'
#preproc_text = features.preprocess([text])
result = new_model.predict(text)
print(result)
#label = labels[result[0]]
#label = labels[result[0].argmax(axis=0)]
#score = ('{:.2f}'.format(round(np.max(result[0]), 2)*100))
#print('LABEL :', label, 'SCORE :', score)

xss
